<a href="https://colab.research.google.com/github/makayla5504/test/blob/main/TF_IDF_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TF-IDF with Scikit-Learn

In our last lab, we started using python for text analysis. Now we will try a text analysis method called *term frequency–inverse document frequency*, often abbreviated *tf-idf*. Tf-idf is a method that tries to identify the most distinctively frequent or significant words in a document.

In this lesson, we're going to learn how to calculate tf-idf scores using a collection of plain text (.txt) files and the Python library scikit-learn, which has a quick and nifty module called [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In this lesson, we will cover how to:
- Calculate and normalize tf-idf scores for U.S. Inaugural Addresses with scikit-learn

## Dataset

### U.S. Inaugural Addresses

<blockquote class="epigraph" style=" padding: 10px">

This is the meaning of our liberty and our creed; why men and women and children of every race and every faith can join in celebration across this magnificent Mall, and why a man whose father less than 60 years ago might not have been served at a local restaurant can now stand before you to take a most sacred oath.  So let us mark this day with remembrance of who we are and how far we have traveled.
<p class ="attribution">—Barack Obama, Inaugural Presidential Address, January 2009
    </p>
    
</blockquote>

During Barack Obama's Inaugural Address in January 2009, he mentioned "women" four different times, including in the passage quoted above. How distinctive is Obama's inclusion of women in this address compared to all other U.S. Presidents? This is one of the questions that we're going to try to answer with tf-idf.

## Breaking Down the TF-IDF Formula

But first, let's quickly discuss the tf-idf formula. The idea is pretty simple.

**tf-idf = term_frequency * inverse_document_frequency**

**term_frequency** = number of times a given term appears in document

**inverse_document_frequency** = log(total number of documents / number of documents with term) + 1**\***

You take the number of times a term occurs in a document (term frequency). Then you take the number of documents in which the same term occurs at least once divided by the total number of documents (document frequency), and you flip that fraction on its head (inverse document frequency). Then you multiply the two numbers together (term_frequency * inverse_document_frequency).

The reason we take the *inverse*, or flipped fraction, of document frequency is to boost the rarer words that occur in relatively few documents. Think about the inverse document frequency for the word "said" vs the word "pigeon." The term "said" appears in 13 (document frequency) of 14 (total documents) *Lost in the City* stories (14 / 13 --> a smaller inverse document frequency) while the term "pigeons" only occurs in 2 (document frequency) of the 14 stories (total documents) (14 / 2 --> a bigger inverse document frequency, a bigger tf-idf boost).

*There are a bunch of slightly different ways that you can calculate inverse document frequency. The version of idf that we're going to use is the [scikit-learn default](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer), which uses "smoothing" aka it adds a "1" to the numerator and denominator:

**inverse_document_frequency**  = log((1 + total_number_of_documents) / (number_of_documents_with_term +1)) + 1

<div class="margin sidebar" style=" padding: 10px">

> If smooth_idf=True (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(t) = log [ (1 + n) / (1 + df(t)) ] + 1.  
> -[scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

</div>

## TF-IDF with scikit-learn

[scikit-learn](https://scikit-learn.org/stable/index.html), imported as `sklearn`, is a popular Python library for machine learning approaches such as clustering, classification, and regression. Though we're not doing any machine learning in this lesson, we're nevertheless going to use scikit-learn's `TfidfVectorizer` and `CountVectorizer`.

Install scikit-learn

In [2]:
!pip install scikit-learn

Import necessary modules and libraries

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
# pd.set_option('max_rows', 600)
pd.set_option('display.max_rows', 600)


from pathlib import Path
import glob

<div class="admonition pandasreview" name="html-admonition" style="background: black; color: white; padding: 10px">
<p class="title">Pandas</p>
 Do you need a refresher or introduction to the Python data analysis library Pandas? Be sure to check out <a href="https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part1.html"> Pandas Basics (1-3) </a> in this textbook!
    
</div>

We're also going to import `pandas` and change its default display setting. And we're going to import two libraries that will help us work with files and the file system: [`pathlib`](https://docs.python.org/3/library/pathlib.html##basic-use) and [`glob`](https://docs.python.org/3/library/glob.html).

#### Set Directory Path

Below we're setting the directory filepath that contains all the text files that we want to analyze.

In [4]:
directory_path = "../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses"


Then we're going to use `glob` and `Path` to make a list of all the filepaths in that directory and a list of all the short story titles.

In [5]:
text_files = glob.glob(f"{directory_path}/*.txt")

In [6]:
text_files

['../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/20_lincoln_1865.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/47_nixon_1973.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/03_adams_john_1797.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/55_bush_george_w_2005.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/45_johnson_1965.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/51_bush_george_h_w_1989.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/05_jefferson_1805.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/49_reagan_1981.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/34_harding_1921.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/42_eisenhower_1953.txt',
 '../content/drive/MyDrive/Colab Notebooks/US_Inaugural_Addresses/08_monroe_1817.txt',
 '../content/d

In [7]:
text_titles = [Path(text).stem for text in text_files]

In [8]:
text_titles

['20_lincoln_1865',
 '47_nixon_1973',
 '03_adams_john_1797',
 '55_bush_george_w_2005',
 '45_johnson_1965',
 '51_bush_george_h_w_1989',
 '05_jefferson_1805',
 '49_reagan_1981',
 '34_harding_1921',
 '42_eisenhower_1953',
 '08_monroe_1817',
 '22_grant_1873',
 '31_taft_1909',
 '28_mckinley_1897',
 '25_cleveland_1885',
 '29_mckinley_1901',
 '46_nixon_1969',
 '21_grant_1869',
 '54_bush_george_w_2001',
 '58_trump_2017',
 '26_harrison_1889',
 '16_taylor_1849',
 '13_van_buren_1837',
 '40_roosevelt_franklin_1945',
 '01_washington_1789',
 '27_cleveland_1893',
 '02_washington_1793',
 '39_roosevelt_franklin_1941',
 '10_adams_john_quincy_1825',
 '38_roosevelt_franklin_1937',
 '44_kennedy_1961',
 '37_roosevelt_franklin_1933',
 '04_jefferson_1801',
 '30_roosevelt_theodore_1905',
 '18_buchanan_1857',
 '52_clinton_1993',
 '14_harrison_1841',
 '33_wilson_1917',
 '35_coolidge_1925',
 '57_obama_2013',
 '50_reagan_1985',
 '12_jackson_1833',
 '11_jackson_1829',
 '07_madison_1813',
 '06_madison_1809',
 '41_tr

## Calculate tf–idf

To calculate tf–idf scores for every word, we're going to use scikit-learn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

When you initialize TfidfVectorizer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf.

The recommended way to run `TfidfVectorizer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in text length, and overall produce more meaningful tf–idf scores. Smoothing and L2 normalization are actually the default settings for `TfidfVectorizer`, so to turn them on, you don't need to include any extra code at all.

Initialize TfidfVectorizer with desired parameters (default smoothing and normalization)

In [9]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')

Run TfidfVectorizer on our `text_files`

In [10]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

Make a DataFrame out of the resulting tf–idf vector, setting the "feature names" or words as columns and the titles as rows

In [11]:
#tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

Add column for document frequency aka number of times word appears in all documents

In [12]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

In [13]:
tfidf_slice = tfidf_df[['government', 'borders', 'people', 'obama', 'war', 'honor','foreign', 'men', 'women', 'children']]
tfidf_slice.sort_index().round(decimals=2)

,government,borders,people,obama,war,honor,foreign,men,women,children
00_Document Frequency,53.00,5.00,56.00,3.00,45.00,32.00,32.00,47.00,15.00,22.00
01_washington_1789,0.11,0.00,0.05,0.00,0.00,0.00,0.00,0.02,0.00,0.00
02_washington_1793,0.06,0.00,0.05,0.00,0.00,0.08,0.00,0.00,0.00,0.00
03_adams_john_1797,0.16,0.00,0.19,0.00,0.01,0.10,0.12,0.04,0.00,0.00
04_jefferson_1801,0.16,0.00,0.01,0.00,0.01,0.04,0.00,0.04,0.00,0.00
05_jefferson_1805,0.03,0.00,0.00,0.00,0.04,0.00,0.06,0.01,0.00,0.02
06_madison_1809,0.00,0.00,0.02,0.00,0.02,0.05,0.05,0.00,0.00,0.00
07_madison_1813,0.04,0.00,0.04,0.00,0.25,0.02,0.02,0.00,0.00,0.00
08_monroe_1817,0.17,0.00,0.11,0.00,0.09,0.01,0.10,0.04,0.00,0.00
09_monroe_1821,0.08,0.00,0.06,0.00,0.11,0.02,0.04,0.01,0.00,0.01


Let's drop "OO_Document Frequency" since we were just using it for illustration purposes.

In [14]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

Let's reorganize the DataFrame so that the words are in rows rather than columns.

In [15]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,20_lincoln_1865,000,0.000000
1,20_lincoln_1865,03,0.031120
2,20_lincoln_1865,04,0.031768
3,20_lincoln_1865,05,0.000000
4,20_lincoln_1865,100,0.000000
...,...,...,...
521937,53_clinton_1997,zachary,0.000000
521938,53_clinton_1997,zeal,0.000000
521939,53_clinton_1997,zealous,0.000000
521940,53_clinton_1997,zealously,0.000000


In [16]:
tfidf_df = tfidf_df.stack().reset_index()

In [17]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

To find out the top 10 words with the highest tf–idf for every story, we're going to sort by document and tfidf score and then groupby document and take the first 10 values.

In [18]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,document,term,tfidf
219683,01_washington_1789,government,0.113681
220084,01_washington_1789,immutable,0.103883
220151,01_washington_1789,impressions,0.103883
222313,01_washington_1789,providential,0.103883
221607,01_washington_1789,ought,0.103728
222327,01_washington_1789,public,0.103102
222093,01_washington_1789,present,0.097516
222365,01_washington_1789,qualifications,0.096372
221787,01_washington_1789,peculiarly,0.090546
216629,01_washington_1789,article,0.085786


In [19]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

We can zoom in on particular words and particular documents.

In [20]:
top_tfidf[top_tfidf['term'].str.contains('women')]

,document,term,tfidf
494862,56_obama_2009,women,0.084859


It turns out that the term "women" is very distinctive in Obama's Inaugural Address.

In [21]:
top_tfidf[top_tfidf['document'].str.contains('obama')]

,document,term,tfidf
486407,56_obama_2009,america,0.148351
491299,56_obama_2009,nation,0.120229
491359,56_obama_2009,new,0.118002
494094,56_obama_2009,today,0.114792
489591,56_obama_2009,generation,0.100654
490763,56_obama_2009,let,0.091100
490579,56_obama_2009,jobs,0.090727
487912,56_obama_2009,crisis,0.087235
489780,56_obama_2009,hard,0.084859
494862,56_obama_2009,women,0.084859


In [22]:
top_tfidf[top_tfidf['document'].str.contains('trump')]

,document,term,tfidf
171442,58_trump_2017,america,0.350162
173623,58_trump_2017,dreams,0.156436
171443,58_trump_2017,american,0.149226
175614,58_trump_2017,jobs,0.142766
177300,58_trump_2017,protected,0.132439
176447,58_trump_2017,obama,0.120288
176804,58_trump_2017,people,0.112370
179039,58_trump_2017,thank,0.109171
172027,58_trump_2017,borders,0.107075
179634,58_trump_2017,ve,0.107075


In [24]:
top_tfidf[top_tfidf['document'].str.contains('kennedy')]

,document,term,tfidf
274787,44_kennedy_1961,let,0.267869
277319,44_kennedy_1961,sides,0.262849
275934,44_kennedy_1961,pledge,0.160960
270645,44_kennedy_1961,ask,0.107713
270877,44_kennedy_1961,begin,0.106495
272004,44_kennedy_1961,dare,0.106495
278908,44_kennedy_1961,world,0.103110
273326,44_kennedy_1961,final,0.102311
275383,44_kennedy_1961,new,0.096600
273133,44_kennedy_1961,explore,0.094223


## Visualize TF-IDF

We can also visualize our TF-IDF results with the data visualization library Altair.

In [23]:
!pip install altair

Let's make a heatmap that shows the highest TF-IDF scoring words for each president, and let's put a red dot next to two terms of interest: "war" and "peace":

The code below was contributed by [Eric Monson](https://github.com/emonson). Thanks, Eric!

In [24]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)

## Your Turn!

I was very cerius about the similarities between inaugural addesses between subsequent years. I just wondered how similar back to back inagural addresses were and a lot ended up being between 20 and 30 percent. I wrote a secetion of code that finds the percent of the similarity between presidents subsequent addresses. For example, how similar is Washingtons first inaugral address to his second inaugral address. The answer was surprisingly low at 8%.

In [25]:
import os


In [30]:

# This def compares the similarties of two texts
def compare_texts(text1, text2):
    # below I used a word overlap to compare how similar two speaches are
    words1 = set(text1.split())
    words2 = set(text2.split())
    overlap = len(words1.intersection(words2))#len counts how many word similarities there are between the two texts

    return overlap / max(len(words1), len(words2))

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:   #I had trouble with this line but the r opens up readand the specifies the encoding as UTF-8 to handle different character encodings
        text = file.read()
    return text

# Load the list of file names
file_names = os.listdir(directory_path)

# Sort the file names in order
file_names.sort()

# Iterate over the file names
for i in range(len(file_names) - 1):
    # Read the text data for the current and next year's addresses
    current_address = read_file(os.path.join(directory_path, file_names[i]))
    next_address = read_file(os.path.join(directory_path, file_names[i + 1]))

    # Compare the similarity between the two addresses
    similarity = compare_texts(current_address, next_address)

  # Print the similarity between the addresses
    print(f"Similarity between {file_names[i]} and {file_names[i + 1]}: {similarity:.2f}")

Similarity between 01_washington_1789.txt and 02_washington_1793.txt: 0.08
Similarity between 02_washington_1793.txt and 03_adams_john_1797.txt: 0.05
Similarity between 03_adams_john_1797.txt and 04_jefferson_1801.txt: 0.25
Similarity between 04_jefferson_1801.txt and 05_jefferson_1805.txt: 0.26
Similarity between 05_jefferson_1805.txt and 06_madison_1809.txt: 0.22
Similarity between 06_madison_1809.txt and 07_madison_1813.txt: 0.28
Similarity between 07_madison_1813.txt and 08_monroe_1817.txt: 0.19
Similarity between 08_monroe_1817.txt and 09_monroe_1821.txt: 0.34
Similarity between 09_monroe_1821.txt and 10_adams_john_quincy_1825.txt: 0.28
Similarity between 10_adams_john_quincy_1825.txt and 11_jackson_1829.txt: 0.19
Similarity between 11_jackson_1829.txt and 12_jackson_1833.txt: 0.30
Similarity between 12_jackson_1833.txt and 13_van_buren_1837.txt: 0.16
Similarity between 13_van_buren_1837.txt and 14_harrison_1841.txt: 0.28
Similarity between 14_harrison_1841.txt and 15_polk_1845.tx

Take a few minutes to explore the dataframe below and then answer the following questions.

**1.** What is the difference between a tf-idf score and raw word frequency?

raw word frequency counts the number of times a term appears in a document. tf-idf score not only counts how maany times a term is in a document but also the terms importance to the whole corpus. tf-idf data is considered more important than raw world frequency which is very straight forward. tf-idf is calculated mybmultiplying the tf and idf scores together.

**2.** Based on the dataframe above, what is one potential problem or limitation that you notice with tf-idf scores?

Sometimes the tf-idf scores had erros in them for example Trumps it-idf score included the word ver which is not even a word and would need to be cleaned.

**Your answer here**

**3.** What's another collection of texts that you think might be interesting to analyze with tf-idf scores?  Why?

td-idf scores would be good to use to analize and catagorizes lists. like if you were big into music and had a really large dataset of music you could catagorize it by artists or genres or year it was released.